In [1]:
import json
import numpy as np
from src.metpyx.definitions import REFERENCE_TEMPERATURE, REFERENCE_PRESSURE
from src.metpyx.ionization_chamber import IonizationChamber, get_radiation_quality_series

# Define monitor ionization chamber
monitor_chamber_id = 'monitor'
monitor = IonizationChamber(identification=monitor_chamber_id, calibrated=False, open_chamber=True)

# Define reference ionization chamber
reference_chamber_id = 'ns557'
reference_chamber_data = '../assets/standard_chambers.json'
with open(reference_chamber_data, 'r') as file:
    json_data = json.load(file)
reference = IonizationChamber(identification=reference_chamber_id, calibrated=True, open_chamber=True, json_data=json_data)

# IR-14D: Calibration of radiation measuring devices
## 1. Initial information
### 1.1. Reference radiation:

In [2]:
# Define radiation quality
radiation_quality = 'L-170'
print(f'Radiation quality: {radiation_quality}\n')

Radiation quality: L-170


### 1.2. Reference chamber:

In [3]:
print(
    f'Identification: {reference_chamber_id}\n'
    # f'Electrometer range: {electrometer_range}\n'
    # f'Electrometer range correction: {electrometer_range_correction}\n'
    f'Data file: {reference_chamber_data}\n'
)

Identification: ns557
Data file: ../assets/standard_chambers.json


### 1.3. Environmental conditions:

In [4]:
print(
    f'Reference temperature: {REFERENCE_TEMPERATURE} ºC\n'
    f'Reference pressure: {REFERENCE_PRESSURE} kPa\n'
)

Environmental conditions:
Reference temperature: 20 ºC
Reference pressure: 101.325 kPa


### 1.4. Other information

In [5]:
print(f'Monitor chamber ID: {monitor_chamber_id}\n')

Monitor chamber ID: monitor


## 2. Simultaneous measurements of reference chamber and monitor chamber
### 2.1. Leakage measurements
Measurements of the monitor chamber

In [6]:
# Readings of the monitor chamber
monitor_leakage_time_readings = np.array([60, 60, 60, 60])
monitor_leakage_charge_readings = np.array([-8.50E-13, -4.30E-13, -2.70E-13, -7.50E-13])

# Define readings units
time_unit = 's'
charge_unit = 'C'

# Measure leakage current with the monitor chamber
monitor_leakage_measurement = monitor.measure_leakage_current(
    time_readings=monitor_leakage_time_readings, charge_readings=monitor_leakage_charge_readings, time_unit=time_unit, charge_unit=charge_unit)

# Export measurements to dataframes
df_monitor_leakage_measurement = monitor_leakage_measurement.to_dataframe()

print(f'{df_monitor_leakage_measurement.to_string(index=True)}\n')

               Time (s)    Charge (C)   Current (A)
#                                                  
Reading 1          60.0 -8.500000e-13 -1.416667e-14
Reading 2          60.0 -4.300000e-13 -7.166667e-15
Reading 3          60.0 -2.700000e-13 -4.500000e-15
Reading 4          60.0 -7.500000e-13 -1.250000e-14
Mean               60.0 -5.750000e-13 -9.583333e-15
Uncertainty         0.0  1.173403e-13  1.955672e-15
% Uncertainty       0.0  2.040701e+01  2.040701e+01


Measurements of the reference chamber

In [7]:
# Readings of the reference chamber
reference_leakage_time_readings = np.array([60, 60, 60, 60, 60])
reference_leakage_charge_readings = np.array([-1.00E-14, -1.10E-13, -1.00E-13, 3.00E-14, 0])

# Measure leakage current with the reference chamber
reference_leakage_measurement = reference.measure_leakage_current(
    time_readings=reference_leakage_time_readings, charge_readings=reference_leakage_charge_readings, time_unit=time_unit, charge_unit=charge_unit)

# Export measurements to dataframes
df_reference_leakage_measurement = reference_leakage_measurement.to_dataframe()
print(f'{df_reference_leakage_measurement.to_string(index=True)}\n')

               Time (s)    Charge (C)   Current (A)
#                                                  
Reading 1          60.0 -1.000000e-14 -1.666667e-16
Reading 2          60.0 -1.100000e-13 -1.833333e-15
Reading 3          60.0 -1.000000e-13 -1.666667e-15
Reading 4          60.0  3.000000e-14  5.000000e-16
Reading 5          60.0  0.000000e+00  0.000000e+00
Mean               60.0 -3.800000e-14 -6.333333e-16
Uncertainty         0.0  2.520317e-14  4.200529e-16
% Uncertainty       0.0  6.632414e+01  6.632414e+01


### 2.2. Air kerma measurements
Measurements of the monitor chamber

In [8]:
# Readings of the monitor chamber
monitor_time_readings = np.array([60, 60, 60, 60, 60])
monitor_pressure_readings = np.array([93.642, 93.642, 93.638, 93.638, 93.633])
monitor_temperature_readings = np.array([20.63, 20.6, 20.6, 20.6, 20.63])
monitor_charge_readings = np.array([4.16E-12, 3.54E-12, 4.18E-12, 4.38E-12, 4.36E-12])
print(f'{df_reference_leakage_measurement.to_string(index=True)}\n')

# Define readings units
time_unit = 's'
charge_unit = 'C'
pressure_unit = 'kPa'
temperature_unit = 'ºC'
current_unit = 'A'

# Measure current with the monitor chamber
monitor_current_measurement = monitor.measure_current(
    time_readings=monitor_time_readings, charge_readings=monitor_charge_readings, time_unit=time_unit,
    charge_unit=charge_unit, background=monitor_leakage_measurement.current.value,
    temperature_readings=monitor_temperature_readings, pressure_readings=monitor_pressure_readings,
    current_unit=current_unit, temperature_unit=temperature_unit, pressure_unit=pressure_unit)

# Export measurements to dataframes
df_monitor_current_measurement = monitor_current_measurement.to_dataframe()

# Print results


               Time (s)    Charge (C)   Current (A)
#                                                  
Reading 1          60.0 -1.000000e-14 -1.666667e-16
Reading 2          60.0 -1.100000e-13 -1.833333e-15
Reading 3          60.0 -1.000000e-13 -1.666667e-15
Reading 4          60.0  3.000000e-14  5.000000e-16
Reading 5          60.0  0.000000e+00  0.000000e+00
Mean               60.0 -3.800000e-14 -6.333333e-16
Uncertainty         0.0  2.520317e-14  4.200529e-16
% Uncertainty       0.0  6.632414e+01  6.632414e+01


Measurements of the reference chamber

In [9]:
# Readings of the reference chamber
reference_time_readings = np.array([60, 60, 60, 60, 60])
reference_pressure_readings = np.array([93.642, 93.642, 93.638, 93.638, 93.633])
reference_temperature_readings = np.array([20.88, 20.91, 20.9, 20.94, 20.96])
reference_charge_readings = np.array([-1.162E-11, -1.168E-11, -1.169E-11, -1.175E-11, -1.164E-11])

# Measure air kerma with the reference chamber
reference_current_measurement = reference.measure_current(
    time_readings=reference_time_readings, charge_readings=reference_charge_readings, time_unit=time_unit,
    charge_unit=charge_unit, background=reference_leakage_measurement.current.value,
    temperature_readings=reference_temperature_readings, pressure_readings=reference_pressure_readings,
    current_unit=current_unit, temperature_unit=temperature_unit, pressure_unit=pressure_unit)
reference_kerma_measurement = reference.measure_air_kerma_rate(
    current_measurement=reference_current_measurement, radiation_quality=radiation_quality)

# Export measurements to dataframes
df_reference_kerma_measurement = reference_kerma_measurement.to_dataframe()

# Print results
print(
    f'Calibration coefficient: {reference.calibration_coefficients[get_radiation_quality_series(radiation_quality)]}\n'
    f'Correction factor: {reference.calibration_coefficients_correction[radiation_quality]}\n'
    f'Distance factor: {0.206378548}\n'  # TODO: Hardcoded value
    f'{df_reference_kerma_measurement.to_string(index=True)}\n'
)

Calibration coefficient: 43700
Correction factor: 0.987
Distance factor: 0.206378548
               Time (s)    Charge (C)   Current (A)  Air kerma (Gy/s)
#                                                                    
Reading 1          60.0 -1.162000e-11 -2.094981e-13     -1.864847e-09
Reading 2          60.0 -1.168000e-11 -2.106048e-13     -1.874699e-09
Reading 3          60.0 -1.169000e-11 -2.107876e-13     -1.876325e-09
Reading 4          60.0 -1.175000e-11 -2.119018e-13     -1.886244e-09
Reading 5          60.0 -1.164000e-11 -2.099371e-13     -1.868755e-09
Mean               60.0 -1.167600e-11 -2.105459e-13     -1.874174e-09
Uncertainty         0.0  2.011964e-14  3.670961e-16      3.267705e-12
% Uncertainty       0.0  1.723162e-01  1.743544e-01      1.743544e-01
